In [ ]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

filename = "hundichat.txt"
raw_text = open(filename, "rb").read()
raw_text = raw_text.decode("utf8")

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print(char_to_int)

n_chars = len(raw_text)
n_vocab = len(chars)

seq_length = 75
step = 23
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, step):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)


X = np.zeros((n_patterns, seq_length, n_vocab), dtype=np.bool)
Y = np.zeros((n_patterns, n_vocab), dtype=np.bool)
        
for i, pattern in enumerate(dataX):
    for t, char in enumerate(pattern):
        X[i, t, char] = 1
    Y[i, dataY[i]] = 1  

In [ ]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1],X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filename = "weights2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [ ]:
import math

def random_sample(X):
    X = X[0]
    X_sq = [value**2 for value in X]    
    value_sum = math.fsum(X_sq)    
    
    rd = np.random.uniform(0, value_sum)
    value_sum = 0
    for i, value in enumerate(X_sq):
        value_sum += value       
        if value_sum >= rd:
            return i

In [ ]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")

for i in range(1000):
    x_pred = np.zeros((1, seq_length, n_vocab))
    for t, char in enumerate(pattern):
        x_pred[0, t, char] = 1.
    prediction = model.predict(x_pred, verbose=0)

    index = random_sample(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
read_in = input("Message:" + "\n")

pattern_str = read_in[len(read_in)-seq_length:len(read_in)]
pattern = list(pattern_str)
pattern = [char_to_int[char] for char in pattern]

print("Response:" + "\n")

for i in range(1000):
    x_pred = np.zeros((1, seq_length, n_vocab))
    for t, char in enumerate(pattern):
        x_pred[0, t, char] = 1.
    prediction = model.predict(x_pred, verbose=0)

    index = random_sample(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]